In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
  root_dir: Path
  test_data_path: Path
  model_path: Path
  metric_file_name: Path
  all_params:dict
  target_column: str

In [6]:
from Phishingproject.constants import *
from Phishingproject.utils.common import read_yaml_file, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        self.schema = read_yaml_file(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import classification_report
from Phishingproject.utils.common import save_txt_file
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_phishing= model.predict(test_x)

        report = classification_report(test_y, predicted_phishing)
        # print(report)
        
        # Saving report as local
        save_txt_file(file_path=Path(self.config.metric_file_name), report=report)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-07-10 16:17:38,248: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-07-10 16:17:38,258: INFO:common: yaml file: params.yaml loaded successfully]
[2024-07-10 16:17:38,266: INFO:common: yaml file: schema.yaml loaded successfully]
[2024-07-10 16:17:38,266: INFO:common: createD Directory at:artifacts]
[2024-07-10 16:17:38,266: INFO:common: createD Directory at:artifacts/model_evaluation]
              precision    recall  f1-score   support

           0       0.76      0.98      0.86      1280
           1       0.97      0.68      0.80      1220

    accuracy                           0.83      2500
   macro avg       0.87      0.83      0.83      2500
weighted avg       0.86      0.83      0.83      2500

